# Exercise 4a - 1D Interpolation



Let’s begin by first importing the function that will be used to perform the interpolation.



```
## interpolate.interp1d()
```
The Python function above belongs to the Scipy package. Since we will use different interpolating functions for each dimensions, we will import ***.interpolate*** from the Scipy library to do the data interpolation. But before that, we need to create a data set that will be used to show the interpolation process.

By defining an x array (using the Numpy function .linspace()) of ten equally spaced numbers ranging from 0 to 100, we will get a list of x with values from 0 to 100 with spaced at 11. (i.e. 0, 10, 20, 30, ... , 100)

For y array, we will use the following equation as a function of x:

# y = 3$x^{2}$ - $e^{0.1x}$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
#defining x and y arrays of the initial data set
x = np.linspace(0, 100,11)
y = 3*x**2 - np.exp(0.1*x)

print("x:")
print(x)

print("y:")
print(y)


Since the process of interpolation allows for obtaining the value of unknown points located within the range of the already known ones, we now define another x array that will contain more points than the first x array (“x”). In particular, we exploit again ***.linspace()*** to build an array of 101 equally spaced numbers "x".

** Q&A: Please modify the code. Can you tell me why we need to use 101 instead of 100?

In [ ]:
# x array that will be used for interpolating new point values
x_new = np.linspace(0, 100, 101)

print(x_new)

There are multiple “kind” options which specifies the type of function that will be used in the interpolating process. Let's print out how different 1D interpolation methods will result different function curve.

In [ ]:
kind = ['linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'previous', 'next']
fig = plt.figure()
ax = fig.subplots()
for i in kind:
      #interpolation step
      f = interpolate.interp1d(x, y, kind = i)
      #y array that contains the interpolated data points
      y_interp = f(x_new)
      ax.plot(x_new, y_interp, alpha = 0.5, label = i)
ax.scatter(x,y)
plt.legend()
plt.show()

## Task 4.1 (Do it together)###

If I have another eqution:  
# **y = 7$x^{3}$ - 3$x^{2}$ + 9$x$**
and I would like to have 1000 spaces on the menu?

- What will the graph looks like?
- Which kind of equation for it?





# Exercise 4b - 2D Interpolation

### Method 1: Interpolate unstructured data

```
# scipy.interpolate.griddata
```

Suppose we want to interpolate the 2-D function on a grid in [0, 1]x[0, 1]

In [15]:
import numpy as np
def func(x, y):
    return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

but we only know its values at 1000 data points

In [16]:
grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]
rng = np.random.default_rng()
points = rng.random((1000, 2))
values = func(points[:,0], points[:,1])

In [17]:
from scipy.interpolate import griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

One can see that the exact result is reproduced by all of the methods to some degree, but for this smooth function the piecewise cubic interpolant gives the best results:

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(221)
plt.imshow(func(grid_x, grid_y).T, extent=(0,1,0,1), origin='lower')
plt.plot(points[:,0], points[:,1], 'k.', ms=1)
plt.title('Original')
plt.subplot(222)
plt.imshow(grid_z0.T, extent=(0,1,0,1), origin='lower')
plt.title('Nearest')
plt.subplot(223)
plt.imshow(grid_z1.T, extent=(0,1,0,1), origin='lower')
plt.title('Linear')
plt.subplot(224)
plt.imshow(grid_z2.T, extent=(0,1,0,1), origin='lower')
plt.title('Cubic')
plt.gcf().set_size_inches(6, 6)
plt.show()

### Method 2: Inverse Distance Weighting Interpolation


Let's interpolate some Velocity Data using IDW method

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Connect Google Drive & Reading .csv files

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
dir = '/content/drive/MyDrive/Colab Notebooks'
drive.mount('/content/drive')

In [27]:
# unstructured triangular grid
data_grid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/grid_point.csv')
data_triangle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/grid_triangle.csv', header=None)

# velocity data
data_velocity = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/velocity.csv')

Visualize the Dataset first:

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))

# unstructured triangular grid
ax.plot(data_grid['x'], data_grid['y'], 'ko', markersize=1, label='grid point (unknown value)')

# velocity data
ax.plot(data_velocity['x'], data_velocity['y'], 'ro', label='velocity data (known value)')

ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')

ax.legend(edgecolor='black', facecolor='white')

plt.show()

Zoom in to observe the data point from x = [260000, 280000] and y =[9000000, 9020000]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

# unstructured triangular grid
ax.plot(data_grid['x'], data_grid['y'], 'ko', markersize=1, label='grid point (unknown value)')

# velocity data
ax.plot(data_velocity['x'], data_velocity['y'], 'ro', label='velocity data (known value)')

ax.set_xlim(260000, 280000)
ax.set_ylim(9000000, 9020000)

ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')

ax.legend(edgecolor='black', facecolor='white')

plt.show()

## Inverse Distance Weighting (IDW)

Step 1: distance calculation

In [ ]:
i = 0 #
nPoints = 5 # number of points
dList = []

for j in range(len(data_velocity)):
    dList.append(distance(data_grid['x'][i],
                          data_velocity['x'][j],
                          data_grid['y'][i],
                          data_velocity['y'][j]))

dListDF = pd.Series(dList)
dListDF.sort_values(ascending=True, inplace=True)
dListDF = dListDF[0:nPoints]

print("dListDF:")
print(dListDF)

print(data_velocity)
print(data_velocity.iloc[dListDF.index.values])

Step 2: Check the closest distance

In [ ]:
if dListDF.iloc[0] <= 1: # if the distance from interpolation point less than equal to 1 meter

    data_velocity['mag'][dListDF.index.values[0]]
    print(True)

else:

    print(False)

Step 3: Perform the IDW

In [ ]:
#=========IDW=========#

# get the velocity data: the closest 5 data

vMag = []
for idx in dListDF.index.values:
    vMag.append(data_velocity['mag'][idx])
vMag = np.array(vMag)

# weight calculation

p = 2                     # power parameter
wList = []
for d in dListDF:
    wList.append(1/d**p)
wList = np.array(wList)

# IDW

np.dot(vMag,np.transpose(wList))/np.sum(wList)

Step 4: Save the result to numpy array

In [47]:
np.savetxt('/content/drive/MyDrive/Colab Notebooks/velocity_IDW.csv', vIDW)

Step 5: Contour Fill Plot

In [ ]:
import matplotlib.tri as mtri
vIDW = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/velocity_IDW.csv')
triang = mtri.Triangulation(data_grid['x'], data_grid['y'], triangles=data_triangle-1)

fig, ax = plt.subplots(figsize=(8,12))
ax.triplot(triang, color='black', linewidth=0.5)
plt.show()

Step 6: Adding axes information

In [ ]:
fig, ax = plt.subplots(figsize=(9,12))

# draw the contour fill
vIDW = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/velocity_IDW.csv')
print(vIDW)
im = ax.triplot(triang, color='black', linewidth=0.5)

# adding the x and y axis labels
ax.set_ylabel('Latitude (UTM)', fontweight='bold', fontsize=12)
ax.set_xlabel('Longitude (UTM)', fontweight='bold', fontsize=12)

# set the x and y ticks
ax.set_yticks(ticks=[9020000, 9040000, 9060000, 9080000, 9100000, 9120000])
ax.set_xticks(ticks=[210000, 230000, 250000, 270000, 290000])

# rotate the yticks label and change the fontsize
ax.set_yticklabels(ax.get_yticks(), rotation=90, fontsize=12, va='center')
ax.set_xticklabels(ax.get_xticks(), fontsize=12, ha='center')

# set the plot axis limit
ax.set_xlim(201000, 305000)
ax.set_ylim(8995900, 9130000)

# colorbar position
cax = fig.add_axes([0.92, 0.15, 0.04, 0.7])

plt.show()